In [1]:
!pip install -qq -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
# Log in to your W&B account
import wandb
from google.colab import userdata
wandb.login(key = userdata.get('WANDB_API_KEY'), verify = True)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True


### Instructions

https://wandb.ai/miteshk/assignments/reports/Assignment-1--VmlldzozNjk4NDE

The goal of this assignment is twofold: (i) implement and use gradient descent (and its variants) with
backpropagation for a classification task (ii) get familiar with wandb which is a cool tool for running and keeping track of a large number of experiments
We strongly recommend that you work on this assignment in a team of size 2. Both the members
of the team are expected to work together (in a subsequent viva both members will be expected to answer questions, explain the code, etc).
Collaborations and discussions with other groups are strictly prohibited.
You must use Python (numpy and pandas) for your implementation.
You cannot use the following packages from keras, pytorch, tensorflow: optimizers, layers
If you are using any packages from keras, pytorch, tensorflow then post on moodle first to check with the instructor.
You can run the code in a jupyter notebook on colab by enabling GPUs.
You have to generate the report in the same format as shown below using wandb.ai. You can start by cloning this report using the clone option above. Most of the plots that we have asked for below can be (automatically) generated using the apis provided by wandb.ai. You will upload a link to this report on gradescope.
You also need to provide a link to your github code as shown below. Follow good software engineering practices and set up a github repo for the project on Day 1. Please do not write all code on your local machine and push everything to github on the last day. The commits in github should reflect how the code has evolved during the course of the assignment.
You have to check moodle regularly for updates regarding the assignment.



###Problem Statement
In this assignment you need to implement a feedforward neural network and write the backpropagation code for training the network. We strongly recommend using numpy for all matrix/vector operations. You are not allowed to use any automatic differentiation packages. This network will be trained and
tested using the Fashion-MNIST dataset. Specifically, given an input image (28 x 28 = 784
pixels) from the Fashion-MNIST dataset, the network will be trained to classify the image
into 1 of 10 classes.

###Question 1 (2 Marks)
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.



In [3]:
from keras.datasets import fashion_mnist
import numpy as np

2024-03-07 07:21:07.535878: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 07:21:07.581103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 07:21:07.581145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 07:21:07.583594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 07:21:07.593250: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 07:21:07.597521: I tensorflow/core/platform/cpu_feature_guard.cc:1

AttributeError: partially initialized module 'keras.src' has no attribute 'utils' (most likely due to a circular import)

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


4422102/4422102 [==============================] - 0s 0us/step


In [4]:
y_train[0]

fmnist_labels = {
0 :	'T-shirt/top',
1 :	'Trouser',
2 :	'Pullover',
3 :	'Dress',
4 :	'Coat',
5 :	'Sandal',
6 :	'Shirt',
7 :	'Sneaker',
8 :	'Bag',
9 :	'Ankle boot'}
fmnist_labels[y_train[0]]

'Ankle boot'

|Label |	Description|
|--- |	---- |
|0 |	T-shirt/top|
|1 |	Trouser|
|2 |	Pullover|
|3 |	Dress|
|4 |	Coat|
|5 |	Sandal|
|6 |	Shirt|
|7 |	Sneaker|
|8 |	Bag|
|9 |	Ankle boot|

In [ ]:
run = wandb.init(
      # Set the project where this run will be logged
      project="pytorch-demo"
      )



In [ ]:
examples = []
for i in range(100):
    pixels = x_train[i]
    image = wandb.Image(pixels, caption=f"{fmnist_labels[y_train[i]]} {i}")
    examples.append(image)
run.log({"examples": examples})


Data cleaning for training it to the neural network

In [10]:

len(np.unique(y_train))

10

In [17]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
one_hot_label = np.zeros([y_train.shape[0], len(np.unique(y_train))], dtype=int)
for index, item in enumerate(y_train):
  one_hot_label[index, item] = 1
print('one_hot_label', one_hot_label)
print('y_train', y_train)


one_hot_label [[0 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
y_train [9 0 0 ... 3 0 5]


###Question 2 (10 Marks)
Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.
Your code should be flexible so that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.
We will check the code for implementation and ease of use.

In [18]:
np.random.rand(1)

array([0.80261302])

In [ ]:
input ---> MNIST Image ----> Hidden Layers --> output ----> Softmax Prob Distribution

In [23]:
import numpy as np


class NeuralNetwork:
    def __init__(self, input_dim, output_dim ,network_layers_architecture: list):

        self.main_model = []
        for index, layer in enumerate(network_layers_architecture):
            self.main_model.append({'weights': [], 'biases': [], 'activation': ''})
            if index == 0:
                self.main_model[index]['weights'] = np.random.rand(layer['num_neurons'], input_dim)
            elif index == len(network_layers_architecture) - 1:
                self.main_model[index]['weights'] = np.random.rand(output_dim, layer['num_neurons'])
            else:
                self.main_model[index]['weights'] = np.random.rand(layer['num_neurons'], layer[index-1]['num_neurons'])
            # self.main_model[index]['activation'] =  layer['activation']
            self.main_model[index]['biases'] = np.random.rand(1)[0]

    def feed_forward(self, input_data):
        print(self.main_model[0]['weights'].shape)
        final_output = np.dot(self.main_model[0]['weights'], input_data) + self.main_model[0]['biases']
        for layer in self.main_model[1:]:
            final_output = np.dot(layer['weights'], final_output) + layer['biases']
        return final_output

    def train(self):
        pass

    def predict(self):
        pass

In [25]:
nn = NeuralNetwork(2, 5, [{'num_neurons': 3, 'activation': 'relu'}, {'num_neurons': 3, 'activation': 'relu'}])

print(nn.feed_forward([1, 2]))


(3, 2)
[3.85876007 4.21436893 2.98569969 2.7165816  2.11141742]




###Question 3 (18 Marks)
Implement the backpropagation algorithm with support for the following optimisation functions
sgd
momentum based gradient descent
nesterov accelerated gradient descent
rmsprop
adam
nadam
(12 marks for the backpropagation framework and 2 marks for each of the optimisation algorithms above)
We will check the code for implementation and ease of use (e.g., how easy it is to add a new optimisation algorithm such as Eve). Note that the code should be flexible enough to work with different batch sizes.



###Question 4 (10 Marks)
Use the sweep functionality provided by wandb to find the best values for the hyperparameters listed below. Use the standard train/test split of fashion_mnist (use (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()).  Keep 10% of the training data aside as validation data for this hyperparameter search. Here are some suggestions for different values to try for hyperparameters. As you can quickly see that this leads to an exponential number of combinations. You will have to think about strategies to do this hyperparameter search efficiently. Check out the options provided by wandb.sweep and write down what strategy you chose and why.
number of epochs: 5, 10
number of hidden layers:  3, 4, 5
size of every hidden layer:  32, 64, 128
weight decay (L2 regularisation): 0, 0.0005,  0.5
learning rate: 1e-3, 1 e-4
optimizer:  sgd, momentum, nesterov, rmsprop, adam, nadam
batch size: 16, 32, 64
weight initialisation: random, Xavier
activation functions: sigmoid, tanh, ReLU
wandb will automatically generate the following plots. Paste these plots below using the "Add Panel to Report" feature. Make sure you use meaningful names for each sweep (e.g. hl_3_bs_16_ac_tanh to indicate that there were 3 hidden layers, batch size was 16 and activation function was ReLU) instead of using the default names (whole-sweep, kind-sweep) given by wandb.




###Question 5 (5 marks)
We would like to see the best accuracy on the validation set across all the models that you train.
wandb automatically generates this plot which summarises the test accuracy of all the models that you tested. Please paste this plot below using the "Add Panel to Report" feature



###Question 6 (20 Marks)
Based on the different experiments that you have run we want you to make some inferences about which configurations worked and which did not.
Here again, wandb automatically generates a "Parallel co-ordinates plot" and a "correlation summary" as shown below. Learn about a "Parallel co-ordinates plot" and how to read it.
By looking at the plots that you get, write down some interesting observations (simple bullet points but should be insightful). You can also refer to the plot in Question 5 while writing these insights. For example, in the above sample plot there are many configurations which give less than 65% accuracy. I would like to zoom into those and see what is happening.
I would also like to see a recommendation for what configuration to use to get close to 95% accuracy.



###Question 7 (10 Marks)
For the best model identified above, report the accuracy on the test set of fashion_mnist and plot the confusion matrix as shown below. More marks for creativity (less marks for producing the plot shown below as it is)



###Question 8 (5 Marks)
In all the models above you would have used cross entropy loss. Now compare the cross entropy loss with the squared error loss. I would again like to see some automatically generated plots or your own plots to convince me whether one is better than the other.



###Question 9 (10 Marks)
Paste a link to your github code for this assignment
Example: \hrefhttps://github.com/<user-id>/cs6910_assignment1https://github.com/<user-id>/cs6910_assignment1;
We will check for coding style, clarity in using functions and a README file with clear instructions on training and evaluating the model (the 10 marks will be based on this)
We will also run a plagiarism check to ensure that the code is not copied (0 marks in the assignment if we find that the code is plagiarised)
We will check the number of commits made by the two team members and then give marks accordingly. For example, if we see 70% of the commits were made by one team member then that member will get more marks in the assignment (\textbfnote that this contribution will decide the marks split for the entire assignment and not just this question).
We will also check if the training and test data has been split properly and randomly. You will get 0 marks on the assignment if we find any cheating (e.g., adding test data to training data) to get higher accuracy



###Question 10 (10 Marks)
Based on your learnings above, give me 3 recommendations for what would work for the MNIST dataset (not Fashion-MNIST). Just to be clear, I am asking you to take your learnings based on extensive experimentation with one dataset and see if these learnings help on another dataset. If I give you a budget of running only 3 hyperparameter configurations as opposed to the large number of experiments you have run above then which 3 would you use and why. Report the accuracies that you obtain using these 3 configurations.
